<a href="https://colab.research.google.com/github/yeialgov/MAYC_ML_Revenue_Prediction/blob/patch-1/Classification%20and%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# download the libraries
import pandas as pd
from pandas import Series,DataFrame
import warnings
from copy import deepcopy
import os
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
import random
%matplotlib inline

# Getting the data

In [48]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [49]:
cd '/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean/'

/content/gdrive/My Drive/ReDi_DA_Project/Datasets_Clean


In [0]:
all_data = pd.read_pickle('cleaned_data.pkl')


# Model to search the visitors who made transactions

## Function to define Target Column

In [0]:
def is_transaction(n):
  if n == 0    :    return 0
  else         :    return 1

all_data['is_transaction']=all_data['totals.transactions'].apply(is_transaction)

In [52]:
all_data['is_transaction'].value_counts()

0    2085062
1      24864
Name: is_transaction, dtype: int64

## Encoding Categorical Values

In [0]:
TARGET = 'is_transaction'
CAT_COLS = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS = [c for c in all_data.columns if c not in CAT_COLS and c != TARGET]


In [0]:
#Columns remove because of problems with the encoder
CAT_COLS.remove('trafficSource.isTrueDirect')
CAT_COLS.remove('trafficSource.adwordsClickInfo.isVideoAd')

In [54]:
CAT_COLS.remove('fullVisitorId')
CAT_COLS

['channelGrouping',
 'customDimension.value',
 'trafficSource.isTrueDirect',
 'geoNetwork.region',
 'trafficSource.adwordsClickInfo.isVideoAd',
 'geoNetwork.city',
 'dataset',
 'trafficSource.source',
 'geoNetwork.metro',
 'device.operatingSystem',
 'totals.visits',
 'trafficSource.medium',
 'trafficSource.keyword',
 'device.browser',
 'geoNetwork.country',
 'geoNetwork.subContinent',
 'geoNetwork.networkDomain',
 'geoNetwork.continent',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'device.deviceCategory',
 'day',
 'month']

In [56]:
CONT_COLS.remove('date')
CONT_COLS.remove('totals.transactions')
CONT_COLS.remove('totals.totalTransactionRevenue')
CONT_COLS

['visitNumber',
 'visitId',
 'totals.newVisits',
 'totals.sessionQualityDim',
 'totals.timeOnSite',
 'device.isMobile',
 'totals.hits',
 'visitStartTime',
 'totals.bounces',
 'totals.pageviews',
 'year',
 'week']

In [57]:
M1_data = deepcopy(all_data[CAT_COLS + CONT_COLS + [TARGET]+ [ 'fullVisitorId']])

M1_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 36 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   channelGrouping                               object 
 1   customDimension.value                         object 
 2   trafficSource.isTrueDirect                    object 
 3   geoNetwork.region                             object 
 4   trafficSource.adwordsClickInfo.isVideoAd      object 
 5   geoNetwork.city                               object 
 6   dataset                                       object 
 7   trafficSource.source                          object 
 8   geoNetwork.metro                              object 
 9   device.operatingSystem                        object 
 10  totals.visits                                 object 
 11  trafficSource.medium                          object 
 12  trafficSource.keyword                         object 
 13

In [67]:
leb = LabelEncoder()
for col in CAT_COLS:
    print(col)
    M1_data[col] = leb.fit_transform(M1_data[col])

channelGrouping
customDimension.value
geoNetwork.region
geoNetwork.city
dataset
trafficSource.source
geoNetwork.metro
device.operatingSystem
totals.visits
trafficSource.medium
trafficSource.keyword
device.browser
geoNetwork.country
geoNetwork.subContinent
geoNetwork.networkDomain
geoNetwork.continent
trafficSource.adwordsClickInfo.adNetworkType
device.deviceCategory
day
month


## Split Train and Test

In [68]:
M1_data['dataset'].value_counts()

1    1708337
0     401589
Name: dataset, dtype: int64

In [0]:
train = M1_data[M1_data['dataset']==1]
test = M1_data[M1_data['dataset']==0]
train.drop('dataset',axis=1,inplace=True)
test.drop('dataset',axis=1, inplace=True)
#remove dataset from the model
CAT_COLS.remove('dataset')

In [0]:
train.shape, test.shape

## Model

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import f1_score as f1
rmse = lambda true, pred: mse(true, pred) ** 0.5

In [0]:
x_train = train[CAT_COLS + CONT_COLS]
x_test = test[CAT_COLS + CONT_COLS]
y_train = train[[TARGET]]
y_test = test[[TARGET]]

In [72]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1708337, 31), (401589, 31), (1708337, 1), (401589, 1))

In [73]:
x_train.columns

Index(['channelGrouping', 'customDimension.value', 'geoNetwork.region',
       'geoNetwork.city', 'trafficSource.source', 'geoNetwork.metro',
       'device.operatingSystem', 'totals.visits', 'trafficSource.medium',
       'trafficSource.keyword', 'device.browser', 'geoNetwork.country',
       'geoNetwork.subContinent', 'geoNetwork.networkDomain',
       'geoNetwork.continent', 'trafficSource.adwordsClickInfo.adNetworkType',
       'device.deviceCategory', 'day', 'month', 'visitNumber', 'visitId',
       'totals.newVisits', 'totals.sessionQualityDim', 'totals.timeOnSite',
       'device.isMobile', 'totals.hits', 'visitStartTime', 'totals.bounces',
       'totals.pageviews', 'year', 'week'],
      dtype='object')

In [0]:
model = RandomForestClassifier( 
    n_estimators=100, # Hyperparameter 1
    max_depth=10,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model = model.fit(x_train, y_train)

In [76]:
def get_error(pred, true):
    print(f'f-score:         {f1(true, pred)}')
    print(f'accuracy:        {accuracy(true, pred)}')

get_error(pred = trained_model.predict(x_test),
          true = y_test)

f-score:         0.1028587846573768
accuracy:        0.9844492752540532


## Submission M1

In [77]:
pred_M1 = trained_model.predict(x_test)
pred_M1.shape

(401589,)

In [78]:
T_predicted = pred_M1[pred_M1!=0]
T_predicted.shape

(656,)

In [0]:
#train_id = train["fullVisitorId"].values
test_id = test["fullVisitorId"].values

In [80]:
sub_M1 = pd.DataFrame({"fullVisitorId":test_id})
sub_M1['PredictedLogRevenue']= pred_M1
sub_M1.shape

(401589, 2)

In [0]:
#sub_M2.append(sub_M1).drop_duplicates('fullVisitorId')

In [81]:
Cus_Pre = sub_M1[sub_M1['PredictedLogRevenue']==1]
test_id_M2 = list(Cus_Pre.index)
len(test_id_M2)

656

In [82]:
sub_M1.drop(test_id_M2,axis=0,inplace=True)
sub_M1.shape

(400933, 2)

In [83]:
sub_M1['PredictedLogRevenue'].value_counts()

0    400933
Name: PredictedLogRevenue, dtype: int64

# Model to calculate transaction Revenue

## Encoding Categorical Values - M2

In [0]:
TARGET_M2 = 'totals.totalTransactionRevenue'
CAT_COLS_M2 = all_data.columns[all_data.dtypes == object].tolist()
CONT_COLS_M2 = [c for c in all_data.columns if c not in CAT_COLS_M2 and c != TARGET_M2]
CAT_COLS_M2.remove('fullVisitorId')

In [0]:
#Columns remove because of problems with the encoder
CAT_COLS_M2.remove('trafficSource.isTrueDirect')
CAT_COLS_M2.remove('trafficSource.adwordsClickInfo.isVideoAd')

In [85]:
M2_data = deepcopy(all_data[CAT_COLS_M2 + CONT_COLS_M2 + [TARGET_M2]+['fullVisitorId']])
M2_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 39 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   channelGrouping                               object        
 1   customDimension.value                         object        
 2   trafficSource.isTrueDirect                    object        
 3   geoNetwork.region                             object        
 4   trafficSource.adwordsClickInfo.isVideoAd      object        
 5   geoNetwork.city                               object        
 6   dataset                                       object        
 7   trafficSource.source                          object        
 8   geoNetwork.metro                              object        
 9   device.operatingSystem                        object        
 10  totals.visits                                 object        
 11  trafficSource.medium     

In [0]:
#leb = LabelEncoder()
for col in CAT_COLS_M2:
    M2_data[col] = leb.fit_transform(M2_data[col])

## Split Train and Test - M2

In [91]:
train_M2 = M2_data[M2_data['dataset']==1]
test_M2 = M2_data[M2_data['dataset']==0]
train_M2.drop('dataset',axis=1,inplace=True)
test_M2.drop('dataset',axis=1, inplace=True)
CAT_COLS_M2.remove('dataset')
train_M2.shape, test_M2.shape

((1708337, 38), (401589, 38))

In [92]:
P_test_M2 = test_M2[test_M2.index.isin(test_id_M2)]
P_test_M2.drop('is_transaction',axis=1,inplace=True)
P_test_M2.shape

(656, 37)

In [93]:
P_test_M2['fullVisitorId'].value_counts()

4099631678878053818    4
4608218212556854907    4
1759300791264456713    3
2521245176378446368    3
9598295196867795955    3
                      ..
7654971282597163377    1
9287412777972824538    1
8006616338750252752    1
7323088264424028056    1
7733537325503662581    1
Name: fullVisitorId, Length: 588, dtype: int64

But the training of the transaction revenue is made just with the data from visitors that made at least one transaction

In [94]:
T_train_M2 = train_M2[train_M2['is_transaction']==1]
T_train_M2.drop('is_transaction',axis=1,inplace=True)
CONT_COLS_M2.remove('date')
CONT_COLS_M2.remove('is_transaction')
T_train_M2.shape, P_test_M2.shape

((18559, 37), (656, 37))

## Model - M2

In [97]:
x_train_M2 = T_train_M2[CAT_COLS_M2 + CONT_COLS_M2]
x_test_M2 = P_test_M2[CAT_COLS_M2 + CONT_COLS_M2]
y_train_M2 = T_train_M2[[TARGET_M2]]
y_test_M2 = P_test_M2[[TARGET_M2]]
x_train_M2.shape, x_test_M2.shape, y_train_M2.shape, y_test_M2.shape

((18559, 32), (656, 32), (18559, 1), (656, 1))

In [102]:
y_train_M2.describe()

,totals.totalTransactionRevenue
count,18559.000000
mean,17.897679
std,1.395750
min,0.000000
25%,17.173452
50%,17.781264
75%,18.504608
max,24.575158


In [0]:
model_M2 = RandomForestRegressor( 
    n_estimators=100, # Hyperparameter 1
    max_depth=10,      # Hyperparameter 2
    random_state=0
)

In [0]:
trained_model_M2 = model_M2.fit(x_train_M2, y_train_M2)

In [104]:
def get_error_regression(pred, true):
    print(f'RMSE:       {rmse(true, pred)}')
    print(f'MAE:        {mae(true, pred)}')
    print(f'r2:         {r2(true, pred)}')

get_error_regression(pred = trained_model_M2.predict(x_test_M2),
          true = y_test_M2)

RMSE:       12.245244617251508
MAE:        8.793990778025604
r2:         -0.7524523523524744


In [105]:
pred_M2 = trained_model_M2.predict(x_test_M2)
pred_M2.shape

(656,)

In [106]:
test_id_M2 = P_test_M2["fullVisitorId"].values
len(test_id_M2) #2896

656

In [107]:
sub_M2 = pd.DataFrame({'fullVisitorId':test_id_M2})
sub_M2['PredictedLogRevenue']= pred_M2
sub_M2

,fullVisitorId,PredictedLogRevenue
0,6533935296546803942,18.194830
1,0855759862361326378,18.029948
2,6915727589933825896,18.353324
3,1180281062741658705,18.375450
4,2721569904173965057,18.330612
...,...,...
651,199211073374659585,18.618384
652,6485327521360234229,18.679087
653,482249714706656154,17.888075
654,9536260586427681418,18.096771


In [108]:
sub_Tot=sub_M2.append(sub_M1)
sub_Tot

,fullVisitorId,PredictedLogRevenue
0,6533935296546803942,18.194830
1,0855759862361326378,18.029948
2,6915727589933825896,18.353324
3,1180281062741658705,18.375450
4,2721569904173965057,18.330612
...,...,...
401584,6701149525099562370,0.000000
401585,6154541330147351453,0.000000
401586,6013469762773705448,0.000000
401587,4565378823441900999,0.000000


# build a submission file

In [0]:
sub_Tot["PredictedLogRevenue"] = np.expm1(sub_Tot["PredictedLogRevenue"])
sub_Tot = sub_Tot.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_Tot.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_Tot["PredictedLogRevenue"] = np.log1p(sub_Tot["PredictedLogRevenue"])
sub_Tot.to_csv("submission.csv", index=False)

In [0]:
sub_Tot.shape

(296530, 2)

In [0]:
sub_Tot[sub_Tot['PredictedLogRevenue']==0]

,fullVisitorId,PredictedLogRevenue
0,0000018966949534117,0.0
1,0000039738481224681,0.0
2,0000073585230191399,0.0
3,0000087588448856385,0.0
4,0000149787903119437,0.0
...,...,...
296525,9999862054614696520,0.0
296526,9999898168621645223,0.0
296527,999990167740728398,0.0
296528,9999915620249883537,0.0


In [0]:
# options for balancing classes
# - upsampling: resampling minority class until it is same quantity of majority class
# - downsampling: removing samples (randomly) from majority class unitl quantity same as minority
# - synthetic (smote): similar to upsampling, algorithmically create new minority samples

# choose a better metric for inbalanced classes:
# - f score

# ensembling methods:
# - classification model: make predictions about whether they made purchase (on test set)
# - regression model: predicted on dataset with no zeros

# options from there:
#  - just use these predictions
# - average out the results from regression model on all data and regression classification combo

In [0]:
from sklearn.metrics import f1_score